#BƯỚC 1: KẾT NỐI VỚI GOOGLE DRIVE

Để có thể load dữ liệu ảnh, lưu file model sau khi huấn luyện, ta phải cài đặt thư viện kết nối Colab với Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# BƯỚC 2: CHUẨN BỊ DỮ LIỆU HUẤN LUYỆN

Bài tập này chúng ta sẽ tiến hành huấn luyện module xác định bounding box khoanh vùng mặt của một con mèo (Cat Face). Tập dữ liệu Cat Dataset bao gồm khoảng 10.000 được download tại trang web sau: https://web.archive.org/web/20150520175645/http://137.189.35.203/WebUI/CatDatabase/catData.html

Dưới đây là một số ảnh nằm trong dataset này.

![Một số ảnh mèo trong dataset](https://web.archive.org/web/20150520175645im_/http://137.189.35.203/WebUI/CatDatabase/Samples.jpg)

**Annotation:**
Dataset gồm có 7 thư mục ảnh 'CAT_00' đến 'CAT_06'. Trong các thư mục ảnh này sẽ có một cặp file có cùng tên khác phần mở rộng tương ứng là ảnh gốc và file chứa các điểm gán nhãn. Ví dụ như cặp file (00000003_015.jpg , 00000003_015.jpg.cat). File .cat sẽ có 9 cặp điểm tương ứng với các bộ phận trên mặt của một con mèo như: đỉnh tai, mắt, mũi, miệng, ...
Lưu ý rằng trong bài toán phát hiện mặt mèo này, chúng ta không cần thông tin của các đỉnh này mà chỉ cần quan tâm đến đường bao (bounding box) xung quanh mặt của một con mèo được tạo dựa trên các đỉnh này.
![cat annotation](https://web.archive.org/web/20150520175645im_/http://137.189.35.203/WebUI/CatDatabase/annotation.png)


In [ ]:
# !wget https://web.archive.org/web/20150520175645/http://137.189.35.203/WebUI/CatDatabase/Data/CAT_DATASET_01.zip
# !unzip CAT_DATASET_01.zip
# !wget https://web.archive.org/web/20150520175645/http://137.189.35.203/WebUI/CatDatabase/Data/CAT_DATASET_02.zip
# !unzip CAT_DATASET_02.zip
# !wget https://web.archive.org/web/20150520175645/http://137.189.35.203/WebUI/CatDatabase/Data/00000003_015.jpg.cat
# !mv 00000003_015.jpg.cat "./CAT_00"
# !mv CAT_* "/content/gdrive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData"

Kiểm tra xem trong thư mục đã có các thư mục dataset chưa?

# BƯỚC 3: KHỞI TẠO THAM SỐ VÀ ĐƯỜNG DẪN

Trong chương trình này chúng ta sẽ sẽ dụng Keras để cài đặt với các tham số mặc định như bên dưới.

In [ ]:
!pip install scipy==1.1.0

In [ ]:
from google.colab import files
files.upload()
files.upload()
files.upload()

Saving dataset.py to dataset.py


Saving split_by_color.py to split_by_color.py


Saving ImageAugmenter.py to ImageAugmenter.py


{'ImageAugmenter.py': b'# -*- coding: utf-8 -*-\n"""Wrapper functions and classes around scikit-images AffineTransformation.\nSimplifies augmentation of images in machine learning.\n\nExample usage:\n        img_width = 32 # width of the images\n        img_height = 32 # height of the images\n        images = ... # e.g. load via scipy.misc.imload(filename)\n\n        # For each image: randomly flip it horizontally (50% chance),\n        # randomly rotate it between -20 and +20 degrees, randomly translate\n        # it on the x-axis between -5 and +5 pixel.\n        ia = ImageAugmenter(img_width, img_height, hlip=True, rotation_deg=20,\n                            translation_x_px=5)\n        augmented_images = ia.augment_batch(images)\n"""\nfrom __future__ import division\nfrom skimage import transform as tf\nimport numpy as np\nimport random\n\ndef is_minmax_tuple(param):\n    """Returns whether the parameter is a tuple containing two values.\n\n    Used in create_aug_matrices() and p

In [ ]:
# !cp  /content/drive/MyDrive/2.SocialTeaching/AdvancedComputerVision/BBR/libraries/ImageAugmenter.py .
# !cp  /content/drive/MyDrive/2.SocialTeaching/AdvancedComputerVision/BBR/libraries/dataset.py .
# # !cp  /content/drive/MyDrive/2.SocialTeaching/AdvancedComputerVision/BBR/libraries/split_by_color.py .

In [ ]:
from __future__ import absolute_import, division, print_function
import sys
CURRENT_DIR = "/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/BT_CatFace"
sys.path.append(CURRENT_DIR);

from dataset import Dataset
import numpy as np
import argparse
import random
import os
from scipy import misc
from skimage import draw
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

np.random.seed(42)
random.seed(42)


MODEL_IMAGE_HEIGHT = 128
MODEL_IMAGE_WIDTH = 128
NB_LOAD_IMAGES = 9500
SPLIT = 0.1 # Tỉ lệ giữa train và validation là 9 : 1
EPOCHS = 150
BATCH_SIZE = 64
SAVE_WEIGHTS_FILEPATH = os.path.join(CURRENT_DIR, "cat_face_locator.weights")
SAVE_WEIGHTS_CHECKPOINT_FILEPATH = os.path.join(CURRENT_DIR, "cat_face_locator.best.weights")
SAVE_PREDICTIONS = True
SAVE_PREDICTIONS_DIR = os.path.join(CURRENT_DIR, "predictions")
DATASET_DIR = "/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData"

# BƯỚC 4: LOAD DỮ LIỆU ẢNH VÀ ANNOTATION

Bài tập này đã chuẩn bị thư viện dataset để phục vụ cho việc load một dataset lên và các thao tác hỗ trợ load theo batch. Nếu bạn quan tâm thì có thể đọc kỹ cài đặt của lớp đối tượng Dataset này.

In [ ]:
!rmdir "/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/.ipynb_checkpoints"
# initialize dataset
subdir_names = ["CAT_00", "CAT_01", "CAT_02", "CAT_03", "CAT_04", "CAT_05", "CAT_06"]
subdirs = [os.path.join(DATASET_DIR, subdir) for subdir in subdir_names]
# dataset = tf.keras.utils.image_dataset_from_directory(DATASET_DIR)
dataset = Dataset(subdirs)

rmdir: failed to remove '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/.ipynb_checkpoints': No such file or directory


In [ ]:
print(subdirs)

['/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_00', '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_01', '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_02', '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_03', '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_04', '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_05', '/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/CatData/CAT_06']


# BƯỚC 5: CHUẨN BỊ MỘT SỐ HÀM CƠ BẢN

Một số hàm cơ bản phục vụ cho quá trình huấn luyện như:
- load_xy: để load toàn bộ dữ liệu và tạo dữ liệu groundtruth dựa trên các điểm đặc trưng trên mặt con mèo.
- unnormalize_prediction: biến đổi vector output đầu ra thành dạng toạ độ ảnh hệ toạ độ nguyên theo pixel. Bước này là để khôi phục hệ toạ độ bounding box từ dạng đã chuẩn hoá (normalized) có miền giá trị 0-1 sang hệ toạ độ theo pixel của ảnh trong tập dữ liệu.
- draw_predicted_rectangle: vẽ bounding box dự đoán từ hệ thống lên ảnh. Bước này bao gồm việc chuẩn hoá ngược sang hệ pixel và vẽ lên trên ảnh màu.
- Clip: để đảm bảo rằng toạ độ đều không âm và không vượt quá khuôn khổ ảnh.



In [ ]:
def load_xy(dataset, nb_load):
    """Loads X and y (examples with labels) for the dataset.
    Examples are images.
    Labels are the coordinates of the face rectangles with their half-heights and half-widths
    (each normalized to 0-1 with respect to the image dimensions.)

    Args:
        dataset            The Dataset object.
        nb_load            Intended number of images to load.
    Returns:
        X (numpy array of shape (N, height, width, 3)),
        y (numpy array of shape (N, 4))
    """
    i = 0
    nb_images = min(nb_load, len(dataset.fps))
    X = np.zeros((nb_images, MODEL_IMAGE_HEIGHT, MODEL_IMAGE_WIDTH, 3), dtype=np.float32)
    y = np.zeros((nb_images, 4), dtype=np.float32)

    for img_idx, image in enumerate(dataset.get_images()):
        if img_idx % 100 == 0:
            print("Loading image %d of %d..." % (img_idx+1, nb_images))
        image.resize(MODEL_IMAGE_HEIGHT, MODEL_IMAGE_WIDTH)
        # Norm image
        X[i] = image.to_array() / 255.0
        # Prepare groundtruth
        face_rect = image.keypoints.get_rectangle(image)
        face_rect.normalize(image)
        center = face_rect.get_center()
        width = face_rect.get_width() / 2
        height = face_rect.get_height() / 2
        y[i] = [center.y, center.x, height, width]
        
        i += 1
        if i >= nb_images:
            break

    return X, y

def unnormalize_prediction(y, x, half_height, half_width, \
                           img_height=MODEL_IMAGE_HEIGHT, img_width=MODEL_IMAGE_WIDTH):
    """Transforms a predictions from normalized (0 to 1) y, x, half-width,
    half-height to pixel values (top left y, top left x, bottom right y,
    bottom right x).
    Args:
        y: Normalized y coordinate of rectangle center.
        x: Normalized x coordinate of rectangle center.
        half_height: Normalized height of rectangle.
        half_width: Normalized width of rectangle.
        img_height: Height of the image to use while unnormalizing.
        img_width: Width of the image to use while unnormalizing.
    Returns:
        (top left y in px, top left x in px, bottom right y in px,
        bottom right x in px)
    """
    # calculate x, y of corners in pixels
    tl_y = int((y - half_height) * img_height)
    tl_x = int((x - half_width) * img_width)
    br_y = int((y + half_height) * img_height)
    br_x = int((x + half_width) * img_width)

    # make sure that x and y coordinates are within image boundaries
    tl_y = clip(0, tl_y, img_height-2)
    tl_x = clip(0, tl_x, img_width-2)
    br_y = clip(0, br_y, img_height-1)
    br_x = clip(0, br_x, img_width-1)

    # make sure that top left corner is really top left of bottom right values
    if tl_y > br_y:
        tl_y, br_y = br_y, tl_y
    if tl_x > br_x:
        tl_x, br_x = br_x, tl_x

    # make sure that the area covered is at least 1px,
    # move preferably the top left corner
    # but dont move it outside of the image
    if tl_y == br_y:
        if tl_y == 0:
            br_y += 1
        else:
            tl_y -= 1

    if tl_x == br_x:
        if tl_x == 0:
            br_x += 1
        else:
            tl_x -= 1

    return tl_y, tl_x, br_y, br_x

def draw_predicted_rectangle(image_arr, y, x, half_height, half_width):
    """Draws a rectangle onto the image at the provided coordinates.
    Args:
        image_arr: Numpy array of the image.
        y: y-coordinate of the rectangle (normalized to 0-1).
        x: x-coordinate of the rectangle (normalized to 0-1).
        half_height: Half of the height of the rectangle (normalized to 0-1).
        half_width: Half of the width of the rectangle (normalized to 0-1).
    Returns:
        Modified image (numpy array)
    """
    #assert image_arr.shape[0] == 3, str(image_arr.shape)
    #height = image_arr.shape[1]
    #width = image_arr.shape[2]
    height = image_arr.shape[0]
    width = image_arr.shape[1]
    tl_y, tl_x, br_y, br_x = unnormalize_prediction(y, x, half_height, half_width, \
                                                    img_height=height, img_width=width)
    image_arr = np.copy(image_arr) * 255
    #image_arr = np.rollaxis(image_arr, 0, 3)
    return draw_rectangle(image_arr, tl_y, tl_x, br_y, br_x)

def draw_rectangle(img, tl_y, tl_x, br_y, br_x):
    """Draws a rectangle onto an image.
    Args:
        img: The image as a numpy array of shape (row, col, channel).
        tl_y: Top left y coordinate as pixel.
        tl_x: Top left x coordinate as pixel.
        br_y: Top left y coordinate as pixel.
        br_x: Top left x coordinate as pixel.
    Returns:
        image with rectangle
    """
    assert img.shape[2] == 3, img.shape[2]
    img = np.copy(img)
    lines = [
        (tl_y, tl_x, tl_y, br_x), # top left to top right
        (tl_y, br_x, br_y, br_x), # top right to bottom right
        (br_y, br_x, br_y, tl_x), # bottom right to bottom left
        (br_y, tl_x, tl_y, tl_x)  # bottom left to top left
    ]
    for y0, x0, y1, x1 in lines:
        rr, cc, val = draw.line_aa(y0, x0, y1, x1)
        img[rr, cc, 0] = val * 255

    return img

def clip(lower, val, upper):
    """Clips a value. For lower bound L, upper bound U and value V it
    makes sure that L <= V <= U holds.
    Args:
        lower: Lower boundary (including)
        val: The value to clip
        upper: Upper boundary (including)
    Returns:
        value within bounds
    """
    if val < lower:
        return lower
    elif val > upper:
        return upper
    else:
        return val



# BƯỚC 6: TẠO DỰNG MÔ HÌNH

Đây là phần việc chính mà các bạn cần thực hiện để cài đặt mô hình phát hiện mặt mèo trong ảnh. Tổng thể sẽ có 4 bước (như slide bài giảng):

Bước 1: Train mô phân lớp đối tượng (hoặc sử dụng pre-trained model)

Bước 2: Nối lớp cuối cùng của lớp Convolution với một mạng Neural để ước lượng (regression) toạ độ bounding box xung quanh mặt một con mèo. Lưu ý đầu ra sẽ là một vector 4 chiều tương ứng với tâm và một nửa bề ngang, bề cao của bounding box.

Bước 3: Huấn luyện mạng Bounding Box Regressor. Lưu ý rằng ta sẽ không huấn luyện các lớp convolution trước đó nữa.

Bước 4: Kết hợp định vị và phân lớp để loại bỏ những ảnh không chữa mèo trong đó.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:

def create_model(image_height, image_width, loss, optimizer):
    """Creates the cat face locator model.

    Args:
        image_height: The height of the input images.
        image_width: The width of the input images.
        loss: Keras loss function (name or object), e.g. "mse".
        optimizer: Keras optimizer to use, e.g. Adam() or "sgd".
    Returns:
        Sequential
    """
    # STEP 1:
    # Load VGG16 model pretrained on ImageNet dataset excluding fully connected layers (Classification Head) (include_top = False)
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))


    # STEP 2: Attach Regression Head into pre-trained convolution layers
    # Freeze the layers except the last 4 layers
    # CODE: for each layer in vgg_conv.layer --> set 'trainable' = False
    
    for layer in vgg_conv.layers[:-4]:
        layer.trainable = False
    for layer in vgg_conv.layers[-4:]:
        vgg_conv.trainable = True


    # Create the model
    model = Sequential()
    # CODE: Add vgg_conv to the model
    # ... model.add(vgg_conv)
    model.add(vgg_conv)
    

    # After adding vgg_conv, the last tensor should be flatten to be a vector
    model.add(Flatten())


    # CODE: Add bounding box Regression Head here.
    # Please choose number of neuron in each layer and suitable activation functions
    # You may use Dropout for generalization
    # Adding 3 layers, 4096 neurons -> ReLU-> 4096 neurons -> ReLU 4 neurons
    # ... model.add(Dense(4096, 'relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation ='relu'))
    model.add(Dense(4, activation='relu'))


    # Compile with mean squared error
    print("Compiling...")
    model.compile(loss=loss, optimizer=optimizer)

    return model


Tiếp theo ta sẽ load dữ liệu và chia ra làm 2 phần: train và validation. Bước này hơi lâu nên cần chạy kiên nhẫn một chút.

In [ ]:
# load images and labels
print("Loading images...")
X, y = load_xy(dataset, NB_LOAD_IMAGES)

# split train and val
nb_images = X.shape[0]
nb_train = int(nb_images * (1 - SPLIT))
X_train = X[0:nb_train, ...]
y_train = y[0:nb_train, ...]
X_val = X[nb_train:, ...]
y_val = y[nb_train:, ...]

Loading images...
Loading image 1 of 9500...


AssertionError: ignored

Bắt đầu, tạo mô hình với độ lỗi sử dụng độ đo L2 và thuật toán Adam Optimizer.

In [ ]:
# You can change loss and optimizer if you want
print("Creating model...")
model = create_model(MODEL_IMAGE_HEIGHT, MODEL_IMAGE_WIDTH, "mse", Adam())


Bắt đầu quá trình huấn luyện và lưu các checkpoint trong quá trình train. Sau khi train hết 150 epoch ta sẽ tiến hành quá trình test trên tập ảnh validation và lưu kết quả vào thư mục Predictions.

In [ ]:
model.summary()

In [ ]:
import cv2

#  STEP 3: Train with Regression Head
# Fit data to model
checkpoint_cb = ModelCheckpoint(SAVE_WEIGHTS_CHECKPOINT_FILEPATH, verbose=1, \
                                save_best_only=True)
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.0,
          validation_data=(X_val, y_val),
          callbacks=[checkpoint_cb])

# save weights
print("Saving weights...")
model.save_weights(SAVE_WEIGHTS_FILEPATH, overwrite=True)

#model.load_weights(SAVE_WEIGHTS_CHECKPOINT_FILEPATH)

# save predictions on val set
if SAVE_PREDICTIONS:
    print("Saving example predictions...")
    y_preds = model.predict(X_val, batch_size=BATCH_SIZE)
    for img_idx, (y, x, half_height, half_width) in enumerate(y_preds):
        img_arr = draw_predicted_rectangle(X_val[img_idx], y, x, half_height, half_width)
        filepath = os.path.join(SAVE_PREDICTIONS_DIR, "%d.png" % (img_idx,))
        cv2.imwrite(filepath, np.squeeze(img_arr))


Sau khi đã huấn luyện xong Bounding Box Regressor. Bạn hãy kết hợp với mạng ban đầu để xác định xem mức độ tin cậy của bounding box tìm được bởi mô hình có mức độ tin cậy là bao nhiêu. Nếu trên ngưỡng thì ta giữ lại, còn lại thì sẽ loại bỏ.

In [ ]:
# Step 4
# CODE: Apply Classification Head and Regression Head for a full system of localization

def check_IoU(img, model, bbox_true, thresh=0.5):
    (tl_y, tl_x, br_y, br_x) = bbox_true
    y_pred = model.predict(np.expand_dims(img, axis=0)) 
    (y, x, half_height, half_width) = y_pred
    (tl_y_pred, tl_x_pred, br_y_pred, br_x_pred) = unnormalize_prediction(y, x, half_height, half_width, img_height=img.shape[0], img_width=img.shape[1])

    xA = max(tl_x, tl_x_pred)
    yA = min(tl_y, tl_y_pred)
    yB = max(br_y, br_y_pred)
    xB = min(br_x, br_x_pred)

    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0

    boxAArea = abs((br_x - tl_x) * (tl_y - br_y))
    boxBArea = abs((br_x_pred - tl_x_pred) * (br_y_pred - tl_y_pred))

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou >= thresh
